# XGBoost.Dask in many threads

Sometimes we want to train many large XGBoost models in parallel.  We do so in this example with ...

1.  The `xgboost.dask` project to do large training runs
2.  Optuna to do hyper-parameter-optimization
3.  A thread pool, to run many of these in parallel
4.  Coiled to launch Dask clusters (but you could swap in your favorite Dask deployment technology as you like)

Using `xgboost.dask` from many threads tooks a couple of small tweaks across projects.  This notebook resulted in the following PRs and issues:

-  https://github.com/dask/distributed/issues/7377
-  https://github.com/dask/dask/pull/9723
-  https://github.com/dask/distributed/pull/7369
-  https://github.com/dmlc/xgboost/pull/8558 (mostly cosmetic, not necessary)
-  Also something in Coiled to allow package_sync to be thread-safe, should be released by 2022-12-07

In [ ]:
FILEPATH="s3://prefect-dask-examples/nyc-uber-lyft/feature_table_fixed_upper_bound.parquet"

In [ ]:
import datetime
import threading
from concurrent.futures import ThreadPoolExecutor

from distributed import Client
import dask.dataframe as dd
from coiled import Cluster
import coiled

import optuna
from dask_ml.metrics import mean_squared_error as lazy_mse
import xgboost as xgb
from xgboost.dask import DaskDMatrix

from dask_ml.datasets import make_classification_df
from dask_ml.model_selection import train_test_split, KFold
from dask_ml.preprocessing import OneHotEncoder
import dask.array as da
import dask.dataframe as dd
from s3fs import S3FileSystem
from xgboost.core import XGBoostError
import numpy as np

import pandas as pd

In [ ]:
import dask, coiled
print("coiled:", coiled.__version__)
print("dask:", dask.__version__)
print("dask.distributed:", dask.distributed.__version__)
print("optuna:", optuna.__version__)
print("xgboost:", xgb.__version__)
print("coiled:", coiled.__version__)

### Train Model

In [ ]:
# Here we subset data for cross-validation

def _make_cv(df, num_folds):
    frac = [1 / num_folds]*num_folds
    splits = df.random_split(frac, shuffle=True)
    for i in range(num_folds):
        train = [splits[j] for j in range(num_folds) if j != i]
        test = splits[i]
        yield train, test


In [ ]:
def train_model(trial_number, study_params, n_splits=5, cluster_name = None, teardown_cluster=False):
    if cluster_name is None:
        thread_id = threading.get_ident()
        cluster_name = "xgb-nyc-taxi-" + str(thread_id)

    cluster = coiled.Cluster(
        worker_vm_types=["m6i.4xlarge"],
        scheduler_vm_types=["m6i.2xlarge"],
        package_sync=True, # copy local packages,
        name=cluster_name,
        shutdown_on_close=False,  # reuse cluster across runs
        show_widget=False,
        n_workers=10,
        use_best_zone=True,
        account="dask-engineering",
        backend_options={"region": "us-east-2", "spot": True, "spot_on_demand_fallback": True}
        )

    print("starting run")
    with Client(cluster) as client:
        print(client.id)
        with client.as_current():
            
            # Load and pre-process the DataFrame
            ddf = dd.read_parquet(FILEPATH)
            ddf = ddf.repartition(partition_size="128MB").persist()
            categorical_vars = ddf.select_dtypes(include="category")
            ddf = ddf.categorize(columns=categorical_vars)
            

            val_scores = []

            
            for i, (train, test) in enumerate(_make_cv(ddf, n_splits)):
                print(f"Starting training run {i}")
                start = datetime.datetime.now()
                train = dd.concat(train)
                try:
                    assert all(train[c].cat.known for c in categorical_vars)
                    assert all(test[c].cat.known for c in categorical_vars)
                except Exception as e:
                    cluster.shutdown()
                    raise RuntimeError(f"Categorical_vars are not known")

                y_train = train['trip_time'].to_frame()
                y_train = y_train.astype(float).persist()
                X_train = train.drop(columns=['trip_time']).persist()

                # Make the training data
                y_test = test['trip_time'].to_frame().persist()
                y_test = y_test.astype(float).persist()
                X_test = test.drop(columns='trip_time').persist()

                try:
                    print("Make dtrain")
                    dtrain = DaskDMatrix(client, X_train, y_train, enable_categorical=True)

                    print("Make dtest")
                    dtest = DaskDMatrix(client, X_test, y_test, enable_categorical=True)

                    print("Training model")

                    model = xgb.dask.train(
                        client,
                        {
                            'verbosity': 2,
                            'tree_method': 'hist', 
                            "objective": "reg:squarederror",
                            **study_params
                        },
                        dtrain,
                        num_boost_round=4,
                        evals=[(dtrain, "train")],
                    )

                    print("Make predictions")
                    # It's faster to run the prediction directly on X_test DataFrame
                    # We also need to confirm that predictions on dtest when it
                    # contains categoricals performs as expected
                    predictions = xgb.dask.predict(client, model, X_test)

                    print("Score the model")
                    score = lazy_mse(y_test.astype("float32").to_dask_array(lengths=True).reshape(-1,), 
                                     predictions.to_dask_array(lengths=True), squared=False,
                                    )
                    print(f"rmse_score:  {score}")
                    val_scores.append(score)
                    print(f"val_scores:  {val_scores}")

                except XGBoostError as e:
                    print(f"Trial {i} failed with {e}")
        return np.mean(val_scores)


In [ ]:
train_options = dict(
    n_splits = 5 
)

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 75, 125),
        'learning_rate': trial.suggest_float('learning_rate', 0.1, 0.6),
        'subsample': trial.suggest_float('subsample', 0, 1),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0, 1),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 3),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0, 1),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0, 1),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 0.5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
    }
    rmse = train_model(
        trial_number=trial.number,
        study_params=params, 
        n_splits=train_options["n_splits"],
    )
    print(f"final mse:  {rmse}")
    return rmse

In [ ]:
# create a single study

study = optuna.create_study(study_name="nyc-taxi-study")

executor = ThreadPoolExecutor(4)

futures = [
    executor.submit(study.optimize, objective, n_trials=3) for _ in range(20)
]

In [ ]:
futures

In [ ]:
study.best_params

In [ ]:
study.best_value

In [ ]:
study.best_trial

In [ ]:
len(study.trials)

In [ ]:
import joblib

In [ ]:
joblib.dump(study, "data/second_study.pkl")

In [ ]:
f = futures[0].result()

In [ ]:
cluster.shutdown()